## Import statements and loading the data

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import collections

In [ ]:
f = h5py.File('bird_spectrograms.hdf5', 'r')
list(f.keys())

In [ ]:
for key in list(f.keys()):
    print(f[key].shape)

In [ ]:
# set seeds to ensure reproducibility 
np.random.seed(5322)
tf.random.set_seed(5322)
random.seed(5322)

## Multiclass Classification (All species)

In [ ]:
X_list = []
y_list = []

# loop through each species' data
for species in list(f.keys()):
    spectrograms = np.array(f[species])  # (128, 517, N)
    n_samples = spectrograms.shape[2]
    labels = [species] * n_samples

    X_list.append(spectrograms)
    y_list.extend(labels)

In [ ]:
# stack and reshape
X = np.concatenate(X_list, axis = 2)      # (128, 517, total_samples)
X = np.transpose(X, (2, 0, 1))           # (samples, 128, 517)
X = X[..., np.newaxis]                   # (samples, 128, 517, 1)

In [ ]:
# labels to arrays
y = np.array(y_list)

In [ ]:
# encode species to ints
le = LabelEncoder()
y_encoded = le.fit_transform(y)          # e.g., 'amecro' → 0, 'amerob' → 1, etc.
label_map = dict(zip(le.transform(le.classes_), le.classes_))
num_classes = len(le.classes_)
num_classes

In [ ]:
# one hot encode
y_onehot = to_categorical(y_encoded)     # shape: (samples, 12)

In [ ]:
# normalize per spectrogram to [0, 1]
X_min = np.min(X, axis = (1, 2, 3), keepdims = True)
X_max = np.max(X, axis = (1, 2, 3), keepdims = True)
X = (X - X_min) / (X_max - X_min + 1e-8)

In [ ]:
# stratified split to get aligned train/test splits for both X, onehot labels, and raw integer labels
X_train, X_test, y_train, y_test, y_train_raw, y_test_raw = train_test_split(
    X, y_onehot, y_encoded, test_size = 0.2, random_state = 5322, stratify = y_encoded
)

In [ ]:
# tried data augmentation by shifting and filling. rotating did not make sense so did not do that
datagen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = False,
    vertical_flip = False,
    fill_mode = 'nearest'
)

datagen.fit(X_train)

In [ ]:
y_train_labels = np.argmax(y_train, axis = 1)

class_weights = class_weight.compute_class_weight(
    class_weight = 'balanced',
    classes = np.unique(y_train_labels),
    y = y_train_labels
)

class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

In [ ]:
# sanity check class distribution and weights
print("Training set class distribution:")
print(collections.Counter(y_train_labels))
print("\nComputed class weights:")
print(class_weights_dict)

# number of classes
print(f"\nNumber of classes: {num_classes}")

In [ ]:
# first multiclass model
model = Sequential([
    # layer group 1
    Conv2D(32, (3, 3), input_shape = (128, 517, 1), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    # group 2
    Conv2D(64, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    # group 3
    Conv2D(64, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    # group 4
    Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.4),
    # group 5
    Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.4),
    # global pooling
    GlobalAveragePooling2D(),
    Dense(units = 128, kernel_regularizer = l2(0.0001)),
    Activation('relu'),
    Dropout(0.5),
    Dense(units = num_classes, activation = 'softmax')
], name = "Multiclass_Model_1")

model.compile(
    optimizer = Adam(learning_rate = 0.0005),
    loss = CategoricalCrossentropy(label_smoothing = 0.05),
    metrics = ['accuracy']
)

model.summary()

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size = 64),
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping],
    class_weight = class_weights_dict
)

# result
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
# predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# get class labels in correct order (by integer encoding)
target_names = [label_map[i] for i in sorted(label_map.keys())]

print(classification_report(y_true, y_pred, target_names = target_names, digits = 4, zero_division = 0))

In [ ]:
# confusion matrix
species_names = list(f.keys())
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize = (12, 10))
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', xticklabels = species_names, yticklabels = species_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.xticks(rotation = 45, ha = 'right')
plt.tight_layout()
plt.show()

This model performs terribly\
Accuracy is low, AUC is low, the classification report shows only two classes (which arent even the majority class) being predicted\
Need to make drastic changes maybe

## Multiclass Classification with adjustments
- Made the model a bit simpler with less layers
- Started at 64 instead of 32 filters
- Removed dropout in first 2 layers to maybe allow more learning since the previous model did not learn much
- Use global max pooling 2d instead of global average pooling 2d since max pooling usually works better than average pooling
- Also reduced batch size to 32 from 64
- Previous model was slow to converge so increased learning rate to 0.001 from 0.0005

In [ ]:
# second multiclass model
model = Sequential([
    # layer group 1
    Conv2D(64, (3, 3), input_shape = (128, 517, 1), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    # group 2
    Conv2D(64, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    # group 3
    Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    # group 4
    Conv2D(256, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    # global pooling
    GlobalMaxPooling2D(),
    Dense(units = 128, kernel_regularizer = l2(0.0001)),
    Activation('relu'),
    Dropout(0.5),
    Dense(units = num_classes, activation = 'softmax')
], name = "Multiclass_Model_2")

model.compile(
    optimizer = Adam(learning_rate = 0.001),
    loss = CategoricalCrossentropy(label_smoothing = 0.05),
    metrics = ['accuracy']
)

model.summary()

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size = 32),
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping],
    class_weight = class_weights_dict
)

# result
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
# predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# get class labels in correct order (by integer encoding)
target_names = [label_map[i] for i in sorted(label_map.keys())]

print(classification_report(y_true, y_pred, target_names = target_names, digits = 4, zero_division = 0))

In [ ]:
# confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize = (12, 10))
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', xticklabels = species_names, yticklabels = species_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.xticks(rotation = 45, ha = 'right')
plt.tight_layout()
plt.show()

## Another Multiclass Model with more Adjustments
- Removed label smoothing since the data may interact with it badly
- Removed l2 to see if it was restricting weights and hurting early learning

In [ ]:
# third multiclass model
model = Sequential([
    # layer group 1
    Conv2D(64, (3, 3), input_shape = (128, 517, 1), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    # group 2
    Conv2D(64, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    # group 3
    Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    # group 4
    Conv2D(128, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    # group 5
    Conv2D(256, (3, 3), kernel_regularizer = l2(0.0001), padding = 'same'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    # global pooling
    GlobalMaxPooling2D(),
    Dense(units = 256, kernel_regularizer = l2(0.0001)),
    Activation('relu'),
    Dropout(0.3),
    Dense(units = num_classes, activation = 'softmax')
], name = "Multiclass_Model_3")

model.compile(
    optimizer = Adam(learning_rate = 0.0005),
    loss = CategoricalCrossentropy(label_smoothing = 0.05),
    metrics = ['accuracy']
)

model.summary()

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size = 64),
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [early_stopping],
    class_weight = class_weights_dict
)

# result
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_acc}')